In [ ]:
import geopandas as gpd
import os
import matplotlib.pyplot as plt


def path_list():
    path_municipios = os.getcwd() + 'municipios.shp'
    path_aeroportos = os.getcwd() + 'aeroportos.shp'
    path_rodovias = os.getcwd() + 'rodovias.shp'

    return [path_municipios, path_aeroportos, path_rodovias]


def open_shapefile():
    return [gpd.read_file(i) for i in path_list()]


def filter_municipios(state, shapes):
    return shapes[shapes['uf'] == state]


def airports(shape, municipio_filtrado):
    gdf = gpd.sjoin(shape, municipio_filtrado, op='within')
    total_airports = gdf.value_counts('TipoAero')
    total_pav = gdf.value_counts('pavimento')

    return gdf, total_airports, total_pav


def roads(shape, municipio_filtrado):
    gdf = gpd.overlay(shape, municipio_filtrado, how='intersection')
    gdf_proj = gdf.copy()
    gdf_proj['geometry'] = gdf_proj['geometry'].to_crs(epsg=5880)
    gdf_proj['distancia km'] = gdf_proj['geometry'].length / 1000
    distancias = gdf_proj.groupby('nm_tipo_tr').agg('sum')['distancia km']

    return gdf, distancias


def run(state):
    shapes = open_shapefile()
    municipio_filter = filter_municipios(state, shapes[0])
    gdf_airports, aeroportos, pavimento = airports(shapes[1], municipio_filter)
    gdf_roads, stats_distance = roads(shapes[2], municipio_filter)
    print(aeroportos)
    print(f'Estatísticas do Estado {state}')
    print(pavimento)
    print(stats_distance)
    gdf_airports.plot()
    gdf_roads.plot()

    return municipio_filter, gdf_airports, gdf_roads

In [ ]:
municipio_filter, gdf_airports, gdf_roads = run('PR')

In [ ]:
municipio_filter['area'] = municipio_filter['geometry'].to_crs(
    epsg=5880).area / 1000000

In [ ]:
municipio_filter.plot(column='area')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
municipio_filter.plot(ax=ax, column='area', legend=True)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
div = make_axes_locatable(ax)
cax = div.append_axes("right", size="5%", pad=0.3)
municipio_filter.plot(ax=ax, column='area', legend=True, cax=cax)

# |
# |
# |
# |

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
plt.title('Área dos Municípios', fontsize=30)
div = make_axes_locatable(ax)
cax = div.append_axes("bottom", size="5%", pad=0.7)
municipio_filter.plot(ax=ax, column='area', cmap='hot_r', legend=True, legend_kwds={
                      'label': 'Área (km2)', 'orientation': 'horizontal'}, cax=cax)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
plt.title('Área dos Municípios', fontsize=30)
municipio.plot(ax=ax, column='area', cmap='copper_r',
               scheme='fisher_jenks_k=10', legend=True)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(1, 1, figsize=(20, 15))
plt.title('Área dos Municípios', fontsize=30, y=1.01)
base = municipio.filter.plot(ax=ax, column='area', cmap='copper_r',
                             scheme='fisher_jenks', k=5, legend=True, legend_kwds={'loc': 'lower right'})
gdf_roads.plot(ax=base, color='Red', linestyle='dashed')
gdf_airport.plot(ax=base, marker="x", color='Blue')
# 'box plot','equal_interval','fisher_jenks','fisher_jenks_sampled',
# 'headtail_breaks','jenks_caspall','jenks_caspall_forced',
# 'jenks_caspall_sampled','max_p_classifier','maximum_breaks',
# 'natural breaks', 'quantiles' ,'percentiles'